In [6]:
import time
import logging

import numpy as np
import h5py

import torch
from torch import optim
from torch.utils.data import DataLoader
from torchsummary import summary
from torch.nn import CrossEntropyLoss

from constants import data_dir
from splice_dataset import SpliceDataset
from splice_ai_torch import SpliceAI, categorical_crossentropy_2d
from utils import get_architecture, SL

from spliceai import SpliceAI as SpliceAIOld

In [7]:
%load_ext autoreload
%autoreload 2

In [8]:
CL, N_GPUS = 80, 1
L = 32

In [9]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [10]:
W, AR, BATCH_SIZE = get_architecture(CL, N_GPUS)

In [ ]:
model_old = SpliceAIOld(L, W, AR)

In [14]:
model_old.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 4)]    0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 32)     160         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, None, 32)     128         conv1d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, None, 32)     0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [11]:
model = SpliceAI(L, W, AR).to(device)
summary(model, input_size=(4, CL + SL), batch_size=BATCH_SIZE)
optimizer = optim.Adam(model.parameters())

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv1d-1             [18, 32, 5080]             160
            Conv1d-2             [18, 32, 5080]           1,056
       BatchNorm1d-3             [18, 32, 5080]              64
              ReLU-4             [18, 32, 5080]               0
            Conv1d-5             [18, 32, 5080]          11,296
       BatchNorm1d-6             [18, 32, 5080]              64
              ReLU-7             [18, 32, 5080]               0
            Conv1d-8             [18, 32, 5080]          11,296
       BatchNorm1d-9             [18, 32, 5080]              64
             ReLU-10             [18, 32, 5080]               0
           Conv1d-11             [18, 32, 5080]          11,296
      BatchNorm1d-12             [18, 32, 5080]              64
             ReLU-13             [18, 32, 5080]               0
           Conv1d-14             [18, 3

In [124]:
h5f = h5py.File(data_dir + 'dataset_train_all.h5', 'r')

num_idx = len(h5f.keys()) // 2
idx_all = np.random.permutation(num_idx)
idx_train = idx_all[:int(0.9 * num_idx)]
idx_valid = idx_all[int(0.9 * num_idx):]

EPOCH_NUM = 10 * len(idx_train)

In [125]:
start_time = time.time()

for epoch_num in range(EPOCH_NUM):

    print(f'On epoch number {epoch_num} / {EPOCH_NUM}.')

    idx = np.random.choice(idx_train)

    X = h5f['X' + str(idx)][:]
    Y = np.asarray(h5f['Y' + str(idx)][:], dtype=np.float32)

    splice_dataset = SpliceDataset(X, Y, CL, N_GPUS)

    dataloader = DataLoader(splice_dataset, batch_size=BATCH_SIZE)
    full_loss = 0
    size = len(dataloader.dataset)
    for batch, (X, y) in enumerate(dataloader):
        pred = model(X)
        loss = categorical_crossentropy_2d(y, pred)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        full_loss += loss.item()

        # if loss.item() > 0.4 and epoch_num > 0:
        #     break
        #
        # if any([y[i].sum(axis=(1,))[0] < y[i].shape[-1] for i in range(y.shape[0])]) and epoch_num > 32:
        #     break
    else:
        print(f"loss: {full_loss:>7f}")
        continue
    break

h5f.close()

On epoch number 0 / 1190.
loss: 1.642687
On epoch number 1 / 1190.
loss: 1.093218
On epoch number 2 / 1190.
loss: 0.014481
On epoch number 3 / 1190.
loss: 0.568272
On epoch number 4 / 1190.
loss: 0.387847
On epoch number 5 / 1190.
loss: 0.016746
On epoch number 6 / 1190.
loss: 0.180956
On epoch number 7 / 1190.
loss: 0.505524
On epoch number 8 / 1190.
loss: 0.229167
On epoch number 9 / 1190.
loss: 0.311344
On epoch number 10 / 1190.
loss: 0.547637
On epoch number 11 / 1190.
loss: 0.199852
On epoch number 12 / 1190.
loss: 0.170929
On epoch number 13 / 1190.
loss: 0.400041
On epoch number 14 / 1190.
loss: 0.011400
On epoch number 15 / 1190.
loss: 0.000982
On epoch number 16 / 1190.
loss: 0.557901
On epoch number 17 / 1190.
loss: 0.738615
On epoch number 18 / 1190.
loss: 0.203652
On epoch number 19 / 1190.
loss: 0.390415
On epoch number 20 / 1190.
loss: 0.537605
On epoch number 21 / 1190.
loss: 0.366669
On epoch number 22 / 1190.
loss: 0.179722
On epoch number 23 / 1190.
loss: 0.029848
On

KeyboardInterrupt: 

In [119]:
pred.shape

torch.Size([18, 3, 5000])

In [120]:
pred[0].shape

torch.Size([3, 5000])

In [121]:
pred[0].sum(axis=(1,))

tensor([4.9999e+03, 3.8468e-02, 5.8596e-02], device='cuda:0',
       grad_fn=<SumBackward1>)

In [122]:
y[0].sum(axis=(1,))

tensor([4988.,    7.,    5.], device='cuda:0')

In [79]:
categorical_crossentropy_2d(y, pred)

tensor(-0., device='cuda:0', grad_fn=<NegBackward>)

In [109]:
CL

80

In [110]:
X = h5f['X' + str(idx)][:]
Y = np.asarray(h5f['Y' + str(idx)][:], dtype=np.float32)

splice_dataset = SpliceDataset(X, Y, CL, N_GPUS)

(668, 5080, 4)
0
0
(668, 5080, 4)
(668, 4, 5080)


In [111]:
splice_dataset.X.shape

(668, 4, 5080)

In [106]:
splice_dataset[34]

(tensor([], device='cuda:0', size=(4, 0)),
 tensor([[1., 1., 1.,  ..., 1., 1., 1.],
         [0., 0., 0.,  ..., 0., 0., 0.],
         [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0'))

In [100]:
h5f['X' + str(idx)][:].shape

(2500, 5080, 4)

In [97]:
y.shape

NameError: name 'x' is not defined

In [98]:
X.shape

torch.Size([18, 4, 0])

In [40]:
y.sum(axis=(0, 2,))[0]

tensor(71945., device='cuda:0')

In [51]:
[y[i].sum(axis=(1,))[0].data for i in range(y.shape[0])]

[tensor(4992., device='cuda:0'),
 tensor(4996., device='cuda:0'),
 tensor(4997., device='cuda:0'),
 tensor(4992., device='cuda:0'),
 tensor(1960., device='cuda:0'),
 tensor(4991., device='cuda:0'),
 tensor(5000., device='cuda:0'),
 tensor(4995., device='cuda:0'),
 tensor(3104., device='cuda:0'),
 tensor(4998., device='cuda:0'),
 tensor(4998., device='cuda:0'),
 tensor(4997., device='cuda:0'),
 tensor(4996., device='cuda:0'),
 tensor(4994., device='cuda:0'),
 tensor(4999., device='cuda:0'),
 tensor(4993., device='cuda:0'),
 tensor(4996., device='cuda:0'),
 tensor(4998., device='cuda:0')]

In [52]:
[y[i].shape for i in range(y.shape[0])]

[torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000]),
 torch.Size([3, 5000])]

In [53]:
y[-1].sum(axis=(1,))[0] < y[-1].shape[-1]

tensor(True, device='cuda:0')

In [54]:
any([y[i].sum(axis=(1,))[0] < y[i].shape[-1] for i in range(y.shape[0])])

True

In [34]:
Y.shape

(1, 1855, 5000, 3)

In [1]:
import wandb

In [2]:
!wandb login

wandb: Currently logged in as: anej (use `wandb login --relogin` to force relogin)
